In [1]:
import pandas as pd
import tensorflow as tf

In [3]:
df = pd.read_csv("./balanced_data2.csv")
df["Rating"].value_counts()

Rating
3    160
4    160
5    160
1    142
2    119
Name: count, dtype: int64

In [4]:
df['Rating'] = df['Rating'] - 1
df["Rating"].value_counts()

Rating
2    160
3    160
4    160
0    142
1    119
Name: count, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.Review.values,
                                                  df.Rating.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.Rating.values)

In [6]:
y_train_series = pd.Series(y_train)
y_val_series = pd.Series(y_val)

train_dist = y_train_series.value_counts()
val_dist = y_val_series.value_counts()

print("train dist:")
print(train_dist)

print("\n val dist:")
print(val_dist)

train dist:
3    136
4    136
2    136
0    120
1    101
Name: count, dtype: int64

 val dist:
4    24
3    24
2    24
0    22
1    18
Name: count, dtype: int64


In [7]:
from transformers import DistilBertTokenizer

checkpoint = "indobenchmark/indobert-base-p2"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint, model_max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [8]:
X_train = list(X_train)
X_val = list(X_val)
y_train = list(y_train)
y_val = list(y_val)

train_tokenized = tokenizer(X_train, padding=True, truncation=True)
val_tokenized = tokenizer(X_val, padding=True, truncation=True)

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokenized),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_tokenized),
))

In [10]:
train_dataset, test_dataset

(<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(434,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(434,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>,
 <_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(268,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(268,), dtype=tf.int32, name=None)},)>)

In [11]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained(checkpoint)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFDistilBertForSequenceClassification: ['mlm___cls', 'bert', 'nsp___cls']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['pre_classifier', 'classifier', 'distilbert', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predict

In [ ]:
history = model.fit(train_dataset,
                    epochs=2,
                    validation_data=test_dataset)